In [1]:
import utils, data, simulation
import os
import os.path as op
import pandas as pd
import numpy as  np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
checkAxon = True
drawAllDist = False

In [3]:
#create an instance of data processing class
data = data.processing()

In [4]:
#folder containing scripts
scriptpath = os.environ['SCRIPT_DATA']
outpath = op.join(scriptpath, 'data')
filename = 'stimulation_thresholds.csv'
df = pd.read_csv(op.join(outpath,'experiment_notes',filename))
df.columns = df.columns.str.strip()

df= df.rename(columns= {'Patient No':'Subject'})


In [5]:
# Remove the ones not used in the double electrode experiment
df = df[df['Selected']!=0]

#change column name
df= df.rename(columns= {'Patient No':'Subject'})
# Average multiple measurements for a given session
avg_data = (df
.groupby(['Subject','Session', 'Electrode'])
.mean())

In [6]:
num_electrode = (df
.groupby(['Subject'])
.nunique())

In [7]:
num_electrode

,Session,Electrode,Threshold,Selected
Subject,,,,
12-005,2,8,7,1
12-104,2,7,10,1
13-101,2,9,9,1


In [8]:
# individual electrodes
df.sort_values(by = ['Threshold'])

,Session,Subject,Electrode,Threshold,Selected
18,2019/05/31,13-101,B10,89,1
29,2019/10/7,13-101,B10,93,1
1,2019/05/29,12-005,A8,153,1
30,2019/10/7,13-101,B9,177,1
25,2019/05/31,13-101,F10,177,1
0,2019/05/29,12-005,A4,194,1
33,2019/10/7,13-101,F10,202,1
20,2019/05/31,13-101,C6,210,1
36,2019/10/8,12-104,B10,217,1
46,2019/10/9,12-005,F2,226,1


In [9]:
medians = (avg_data
.groupby(['Subject'])
.median())
medians.index

Index(['12-005', '12-104', '13-101'], dtype='object', name='Subject')

In [10]:
t_table = pd.DataFrame()
for subject in medians.index:
    query_string = 'Subject==' + '"' + subject + '"'
    print(query_string)  
    # select a single subject
    selected = avg_data.query(query_string)

    #calculate interquartile range of values in the 'points' column
    median = selected['Threshold'].median()
    q75, q25 = np.percentile(selected['Threshold'], [75 ,25])
    iqr = q75 - q25
    t_table = t_table.append({'Subject': subject, 'Median': median, 'Q25-Q75': (q25, q75),
                              'IQR': iqr, 'Minimum': np.min(selected['Threshold']), 
                             'Maximum': np.max(selected['Threshold'])}, ignore_index=True)

Subject=="12-005"
Subject=="12-104"
Subject=="13-101"


In [11]:
t_table

,IQR,Maximum,Median,Minimum,Q25-Q75,Subject
0,97.0,484.0,274.0,153.0,"(226.0, 323.0)",12-005
1,241.5,645.0,475.5,217.0,"(371.0, 612.5)",12-104
2,93.0,323.0,210.0,89.0,"(177.0, 270.0)",13-101


## Catch statistics

In [12]:
filename = 'S_12-005_12-104_13-101__collated_both_sessions.csv'

(double_x2, catch, subjectdata) = data.get_two_point(outpath, scriptpath, filename)

catch_means = pd.DataFrame()
for subject in medians.index:
    query_string = 'subject_id==' + '"' + subject + '"'
    print(query_string)  
    #select subject
    split = catch.query(query_string)
    # total number of trials per participant
    n= len(split)
    # times they perceived nothing when no stim
    hit = len(split[split['pts_number_processed']==0])
    # times they perceived >0 shapes
    fa = len(split[split['pts_number_processed']>0])
    print(n)
    print(hit)
    print(fa)
    catch_means = catch_means.append( {'subject_id': subject, 'false alarm rate': fa,
                             'total no of trials' : n, 'fa/n' : fa/n }, 
                             ignore_index= True)


subject_id=="12-005"
15
15
0
subject_id=="12-104"
12
10
2
subject_id=="13-101"
16
16
0


In [13]:
catch_means

,fa/n,false alarm rate,subject_id,total no of trials
0,0.000000,0.0,12-005,15.0
1,0.166667,2.0,12-104,12.0
2,0.000000,0.0,13-101,16.0
